<a href="https://colab.research.google.com/github/jozijoseph507-lab/CareerFlow-API/blob/main/AlgoVisualizer_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ---------------- Install dependencies ----------------
!pip install fastapi uvicorn nest-asyncio

# ---------------- Imports ----------------
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List, Optional
import nest_asyncio
import uvicorn
from threading import Thread
import time
import requests
import os

# Kill previous server on port 8000
os.system("kill $(lsof -t -i:8000) 2>/dev/null")

# Fix nested async loop for Colab
nest_asyncio.apply()

# ---------------- FastAPI app ----------------
app = FastAPI(title="AlgoVisualizer API")

# ---------------- Models ----------------
class AlgorithmRequest(BaseModel):
    array: List[int]
    target: Optional[int] = None  # for search algorithms

class AlgorithmStep(BaseModel):
    step: int
    array_state: List[int]

class AlgorithmResponse(BaseModel):
    algorithm: str
    steps: List[AlgorithmStep]
    time_complexity: str
    notes: List[str]

# ---------------- Algorithms ----------------
def bubble_sort_steps(arr):
    steps = []
    array = arr.copy()
    for i in range(len(array)):
        for j in range(0, len(array)-i-1):
            if array[j] > array[j+1]:
                array[j], array[j+1] = array[j+1], array[j]
                steps.append(AlgorithmStep(step=len(steps)+1, array_state=array.copy()))
    return steps

def merge_sort_steps(arr):
    steps = []
    def merge(left, right):
        result = []
        while left and right:
            if left[0] <= right[0]:
                result.append(left.pop(0))
            else:
                result.append(right.pop(0))
        result += left + right
        return result
    def recursive_sort(array):
        if len(array) <= 1:
            return array
        mid = len(array)//2
        left = recursive_sort(array[:mid])
        right = recursive_sort(array[mid:])
        merged = merge(left, right)
        steps.append(AlgorithmStep(step=len(steps)+1, array_state=merged.copy()))
        return merged
    recursive_sort(arr.copy())
    return steps

def binary_search_steps(arr, target):
    steps = []
    array = sorted(arr.copy())
    low, high = 0, len(array)-1
    found = False
    while low <= high:
        mid = (low + high)//2
        steps.append(AlgorithmStep(step=len(steps)+1, array_state=array[low:high+1]))
        if array[mid] == target:
            found = True
            break
        elif array[mid] < target:
            low = mid + 1
        else:
            high = mid - 1
    return steps, found

# ---------------- API Endpoints ----------------
@app.post("/visualize/sort/bubble", response_model=AlgorithmResponse)
def api_bubble_sort(request: AlgorithmRequest):
    notes = []
    if not request.array:
        notes.append("Empty array provided.")
    if len(request.array) != len(set(request.array)):
        notes.append("Array contains duplicate values.")
    steps = bubble_sort_steps(request.array)
    return AlgorithmResponse(
        algorithm="Bubble Sort",
        steps=steps,
        time_complexity="O(n^2)",
        notes=notes
    )

@app.post("/visualize/sort/merge", response_model=AlgorithmResponse)
def api_merge_sort(request: AlgorithmRequest):
    notes = []
    if not request.array:
        notes.append("Empty array provided.")
    if len(request.array) != len(set(request.array)):
        notes.append("Array contains duplicate values.")
    steps = merge_sort_steps(request.array)
    return AlgorithmResponse(
        algorithm="Merge Sort",
        steps=steps,
        time_complexity="O(n log n)",
        notes=notes
    )

@app.post("/visualize/search/binary", response_model=AlgorithmResponse)
def api_binary_search(request: AlgorithmRequest):
    notes = []
    if not request.array:
        notes.append("Empty array provided.")
    if request.target is None:
        notes.append("No target provided.")
        return AlgorithmResponse(algorithm="Binary Search", steps=[], time_complexity="O(log n)", notes=notes)
    steps, found = binary_search_steps(request.array, request.target)
    if not found:
        notes.append(f"Target {request.target} not found.")
    if len(request.array) != len(set(request.array)):
        notes.append("Array contains duplicate values.")
    return AlgorithmResponse(
        algorithm=f"Binary Search for {request.target}",
        steps=steps,
        time_complexity="O(log n)",
        notes=notes
    )

# ---------------- Run FastAPI in background thread ----------------
def run_app():
    uvicorn.run(app, host="127.0.0.1", port=8000)

thread = Thread(target=run_app, daemon=True)
thread.start()
time.sleep(2)  # Give server time to start
print("FastAPI server is running in Colab!")

# ---------------- Example Requests ----------------
print("\nBubble Sort Result:")
print(requests.post("http://127.0.0.1:8000/visualize/sort/bubble", json={"array":[5,2,9,1]}).json())

print("\nMerge Sort Result:")
print(requests.post("http://127.0.0.1:8000/visualize/sort/merge", json={"array":[5,2,9,1]}).json())

print("\nBinary Search Result:")
print(requests.post("http://127.0.0.1:8000/visualize/search/binary", json={"array":[5,2,9,1], "target":9}).json())
